# Setup

In [2]:
import requests
import json
import pandas as pd
import duckdb

def api_request(endpoint):
    url = f'https://pokeapi.co/api/v2/{endpoint}/'
    response = requests.get(url)
    data = response.json()
    return data

def get_gen_poke(gen):
    data = api_request(f'pokedex/{gen}')
    df = pd.json_normalize(data['pokemon_entries'])
    return df

def get_region_locations(name):
    data = api_request(f'region/{name}')
    df = pd.json_normalize(data['locations'])
    return df

# Main

In [2]:
# 1. Get pokemon from emerald generation
poke_entries = get_gen_poke('hoenn')
poke_entries['id'] = poke_entries['pokemon_species.url'].apply(lambda x: x.split('pokemon-species/')[1].split('/')[0])


In [3]:
# 2. Get emerald region locations
locs = get_region_locations('hoenn')

In [ ]:
# Import data into duckdb
try:
    con = duckdb.connect()
    con.execute("CREATE TABLE pokedex AS SELECT * FROM poke_entries")
finally:
    con.close()

# Debug

In [ ]:
locs

In [ ]:
poke_entries

In [2]:
# Get location encounter data for each location in the Hoenn region
data = api_request('location-area/429')
# Treat the data
poke_encounters = pd.json_normalize(
    data['pokemon_encounters'],
    record_path = 'version_details',
    meta = [['pokemon', 'name'], ['pokemon', 'url']]
).explode('encounter_details')
encounter_details = poke_encounters['encounter_details'].apply(pd.Series)

In [46]:
import aiohttp
import asyncio

async def get_url(session, url):
    async with session.get(url) as response:
        if response.status == 404:
            return None
        return await response.json()

async def get_location_encounters():
    async with aiohttp.ClientSession() as session:
        tasks = [get_url(session, url) for url in df_location_data['areas.url']]
        results = await asyncio.gather(*tasks)

        return results

async def get_location_info():
    async with aiohttp.ClientSession() as session:
        tasks = [get_url(session, url) for url in locs['url']]
        results = await asyncio.gather(*tasks)

        return results

location_data = await get_location_info()
df_location_data = pd.json_normalize(
    data=location_data,
    record_path='areas',
    meta=['id', 'name'],
    meta_prefix='location.',
    record_prefix='areas.'
)

In [49]:
location_encounters = await get_location_encounters()

In [ ]:
pd.DataFrame(location_encounters)

In [ ]:
df = pd.json_normalize(
    data=location_encounters,
    record_path=['pokemon_encounters', 'version_details'],
    meta='id'
)
df[df['version.name'] == 'emerald']

In [ ]:
pd.json_normalize(
    data=location_encounters,
    record_path=['pokemon_encounters', 'version_details', 'encounter_details'],
    meta=['id', 'name', 'pokemon_encounters', 'location']
)